In [1]:
from modelUnetVGG16 import *
from data import *

vid="v026" #version id should match the file number 
# Ran on DellWS with GeForce RTX3060 GPU

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=2
learning_rate=1e-4

train_gen = trainGenerator(batch_size,'mass_seg_05/train05','mass_seg','mass_seg_labels',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'mass_seg_05/valid05','mass_seg','mass_seg_labels',data_gen_args,save_to_dir = None)

# 300 images are used for training, 52 images for validating, and 60 images for testing
train_steps = 300//batch_size
valid_steps = 52//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=3*train_steps
num_epochs=100
#num_top_filter=16

model = build_vgg16_unet()

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files05/unet_mass_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files05/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Found 300 images belonging to 1 classes.
Found 300 images belonging to 1 classes.
Epoch 1/100
450/450 [==============================] - ETA: 0s - loss: 0.8324 - acc: 0.9244 - dice_coef: 0.1676 - iou: 0.0991Found 52 images belonging to 1 classes.
Found 52 images belonging to 1 classes.
450/450 [==============================] - 243s 488ms/step - loss: 0.8324 - acc: 0.9244 - dice_coef: 0.1676 - iou: 0.0991 - val_loss: 0.9146 - val_acc: 0.9744 - val_dice_coef: 0.0854 - val_iou: 0.0464

Epoch 00001: saving model to files05\unet_mass_seg_v026.hdf5
Epoch 2/100
450/450 [==============================] - 223s 496ms/step - loss: 0.7409 - acc: 0.9668 - dice_coef: 0.2591 - iou: 0.1600 - val_loss: 0.8496 - val_acc: 0.9795 - val_dice_coef: 0.1504 - val_iou: 0.0899

Epoch 00002: saving model to files05\unet_mass_seg_v026.hdf5
Epoch 3/100
450/450 [==============================] - 219s 487ms/step - loss: 0.6804 - acc: 0.9734 - dice_coef: 0.3196 - iou: 0.2071 - val_loss: 0.7119 - val_acc: 0.9807 - va

### validate your model and save predicted results

In [3]:
validGene = testGenerator("mass_seg_05/valid05/pred")
model.load_weights("files05/unet_mass_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,52,verbose=1)
saveResult("mass_seg_05/valid05/pred",results,vid)

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


52/52 [==============================] - 23s 360ms/step


D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\0_predict_v026.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\1_predict_v026.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\3_predict_v026.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\7_predict_v026.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_05/valid05/pred\12_predict_v026.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png